# Import & Constants

In [1]:
import itertools
import matplotlib.pyplot as plt
import numpy as np
import os
from graph_tool.all import *

In [2]:
C = 2
L = 1
probabilityLayer = ["A", "B"]

# Probability Calculator & Cell Generator

In [11]:
def Combinations_Generator(C,L):# Generate all possible combinations states
    N = C*(L+1)
    all_combinations = np.array(list(itertools.product([0, 3], repeat=N)))
    return all_combinations

In [4]:
def Combination_Probability(L, C, combination, probabilityLayer):
    
    combinationProbability = ""
    c = 0
    i = 0
        
    for edge in combination:
        if(edge == 1):
            combinationProbability += f"p{probabilityLayer[i]}1 *"
        elif(edge == 2):
            combinationProbability += f"p{probabilityLayer[i]}2 *"
        elif(edge == 3):
            combinationProbability += f"p{probabilityLayer[i]}3 *"
        else:
            combinationProbability += f"p{probabilityLayer[i]}0 *"

        c += 1
        if(c%C == 0):
            i += 1
        combinationProbability = combinationProbability[:-1]

    return combinationProbability

In [5]:
#probabilityLayer = ["p", "p"]
#for combination in Combinations_Generator(C,L):
#    print(Combination_Probability(L, C, combination, probabilityLayer))

In [5]:
def Vertex_Generator(g, a, b, comb):
    if(comb == 1):
        g.add_edge(g.vertex_index[a], g.vertex_index[b])
        
    elif(comb == 2):
        g.add_edge(g.vertex_index[b],g.vertex_index[a])
        
    elif(comb == 3):
        g.add_edge(g.vertex_index[a], g.vertex_index[b])
        g.add_edge(g.vertex_index[b],g.vertex_index[a])
        
    return

In [6]:
def Cell_Generator(L,C, comb):
    
    g = Graph()
    g.add_vertex(L*C + 2)
    
    l = 0
    
    for i in range(L):
        if(i==0):
            for j in range(C):
                Vertex_Generator(g, i, j+1, comb[l])
                l += 1
        else:
            for j in range(C):
                Vertex_Generator(g, i+j, i+j+C, comb[l])
                l += 1
    
    for j in range(C):
        Vertex_Generator(g, (L*C-C)+1+j, L*C+1, comb[l])
        l += 1
    
    return g

In [7]:
def Percolation_Check(compDown, compUp):
    
    aDown = compDown.a[0]
    bDown = compDown.a[-1]

    aUp = compUp.a[0]
    bUp = compUp.a[-1]
    if(aDown == bDown & aUp == bUp):
        return int(3)
    elif(aDown == bDown):
        return int(1)
    elif(aUp == bUp):
        return int(2)
    return int(0)

In [8]:
def Group_Probability_Type(L, C, combination, probabilityLayer, percolationType, probabilityType0, probabilityType1, probabilityType2, probabilityType3):
    
    probability = Combination_Probability(L, C, combination, probabilityLayer)
    
    if(percolationType == 0):
        probabilityType0 += f"{probability} +"
        
    elif(percolationType == 1):
        probabilityType1 += f"{probability} +"
    
    elif(percolationType == 2):
        probabilityType2 += f"{probability} +"

    elif(percolationType == 3):
        probabilityType3 += f"{probability} +"

    else:
        return "ERROR"
        
    return probabilityType0, probabilityType1, probabilityType2, probabilityType3

In [12]:
probabilityType0 = "" 
probabilityType1 = "" 
probabilityType2 = ""
probabilityType3 = ""
for combination in Combinations_Generator(C,L):
    g = Cell_Generator(L,C, combination)
    compDown = graph_tool.topology.label_out_component(g, 0, label=None)
    compUp = graph_tool.topology.label_out_component(g, L*C+1, label=None)
    percolationType = Percolation_Check(compDown, compUp)
    probabilityType0, probabilityType1, probabilityType2, probabilityType3 = Group_Probability_Type(L, C, combination, probabilityLayer, percolationType, probabilityType0, probabilityType1, probabilityType2, probabilityType3)

In [13]:
#probabilityType0

In [15]:
#probabilityType1

In [29]:
#probabilityType2

In [13]:
probabilityType3

'pA0 pA3 pB0 pB3  +pA0 pA3 pB3 pB3  +pA3 pA0 pB3 pB0  +pA3 pA0 pB3 pB3  +pA3 pA3 pB0 pB3  +pA3 pA3 pB3 pB0  +pA3 pA3 pB3 pB3  +'

## Generate Graphs

In [14]:
for combination in Combinations_Generator(C,L):
    g = Cell_Generator(L,C, combination)
    compDown = graph_tool.topology.label_out_component(g, 0, label=None)
    compUp = graph_tool.topology.label_out_component(g, L*C+1, label=None)
    percolationType = Percolation_Check(compDown, compUp)
    if(percolationType == 1):
        graph_draw(g, vertex_text=g.vertex_index, output = f"Graphs/Type{percolationType}/{combination}.png")
    elif(percolationType == 2):
        graph_draw(g, vertex_text=g.vertex_index, output = f"Graphs/Type{percolationType}/{combination}.png")
        
    elif(percolationType == 3):
        graph_draw(g, vertex_text=g.vertex_index, output = f"Graphs/Type{percolationType}/{combination}.png")
    else:
        graph_draw(g, vertex_text=g.vertex_index, output = f"Graphs/Type{percolationType}/{combination}.png")

In [15]:
percolationType = np.zeros(len(Combinations_Generator(C,L)))
i = 0
for combination in Combinations_Generator(C,L):
    g = Cell_Generator(L,C, combination)
    compDown = graph_tool.topology.label_out_component(g, 0, label=None)
    compUp = graph_tool.topology.label_out_component(g, L*C+1, label=None)
    percolationType[i] = Percolation_Check(compDown, compUp)
    
    i += 1
    #print(Percolation_Check(compDown, compUp))
    #print(Combination_Probability(L, C, combination, probabilityLayer))

uniqueType, counts = np.unique(percolationType, return_counts=True)

# Combine unique elements and counts into a dictionary
typeCounts = dict(zip(uniqueType, counts))

# Print the result
print("Element counts:", typeCounts)

Element counts: {0.0: 9, 3.0: 7}


# Cluster Mass

In [16]:
def Edge_Vertex_Relation():#Returns a matrix that relates the edges with the vertex
    L = 1
    C = 2
    relation_list = []
    
    for i in range(L):
        if(i==0):
            for j in range(C):
                relation_list.append((i, j+1))
        else:
            for j in range(C):
                relation_list.append((i+j, i+j+C))
                
    for j in range(C):
        relation_list.append(((L*C-C)+1+j, L*C+1))
    return np.array(relation_list)

In [17]:
def Common_Elements(array1, array2):
    set1 = set(array1)
    set2 = set(array2)

    commonElements = set1.intersection(set2)
    numberCommonElements = len(commonElements)

    if numberCommonElements == 0:
        return 0
    elif numberCommonElements == 1:
        return 1
    else:
        return 2

In [18]:
def Above_Below(array1, array2):
    
    set1 = set(array1)
    set2 = set(array2)
    commonElements = set1.intersection(set2)
    
    if max(set1) in commonElements:
        # If the maximum element of set1 is in common with set2, return 1
        return 1
    else:
        # Otherwise, return 0
        return 0

In [19]:
def Find_Vertex(arr):#Return all vertex index that belongs to the cell cluster SCC
    return [index for index, value in enumerate(arr) if value == 1]

In [20]:
def Find_Vertex_Edges(targetVertex):#Return the edges index related to the target vertex
    indices = [index for index, line in enumerate(Edge_Vertex_Relation()) if targetVertex in line]
    return indices

In [21]:
def Same_SCC(arr, targetIndex):
    
    value = arr[targetIndex]
    sameIndexValue = np.where(arr == value)[0]
    
    return sameIndexValue

In [22]:
def Cell_SCC(g):
    compDisconnected = graph_tool.topology.label_components(g, attractors=True)
    Vertex_Generator(g, 0, L*C+1, 3)
    compConnected = graph_tool.topology.label_components(g, attractors=True)
    return compDisconnected, compConnected

In [23]:
def Edge_Mass_Contribution(edgeVertexRelation, vertexIndexSCC, combination, probabilityLayer):
    edgeMass = ""
    
    if(Common_Elements(edgeVertexRelation, vertexIndexSCC) == 1 and combination==0):
        if(Above_Below(edgeVertexRelation, vertexIndexSCC)==0):
            edgeMass += f"t{probabilityLayer}top +"
        else:
            edgeMass += f"t{probabilityLayer}bottom +"
                
    elif(Common_Elements(edgeVertexRelation, vertexIndexSCC) == 1 and combination==1):
        if(Above_Below(edgeVertexRelation, vertexIndexSCC)==0):
            edgeMass += f"mi1{probabilityLayer}top +"
        else:
            edgeMass += f"mi1{probabilityLayer}bottom +"

    elif(Common_Elements(edgeVertexRelation, vertexIndexSCC) == 1 and combination==2):
        if(Above_Below(edgeVertexRelation, vertexIndexSCC)==0):
            edgeMass += f"mi2{probabilityLayer}top +"
        else:
            edgeMass += f"mi2{probabilityLayer}bottom +"
            
    elif(Common_Elements(edgeVertexRelation, vertexIndexSCC) == 2 and combination==0):
        edgeMass += f"t{probabilityLayer}top +t{probabilityLayer}bottom +"
            
    elif(Common_Elements(edgeVertexRelation, vertexIndexSCC) == 2 and combination==1):
        edgeMass += f"m1{probabilityLayer} +"

    elif(Common_Elements(edgeVertexRelation, vertexIndexSCC) == 2 and combination==2):
        edgeMass += f"m2{probabilityLayer} +"
        
    elif(Common_Elements(edgeVertexRelation, vertexIndexSCC) == 2 and combination==3):
        edgeMass += f"M{probabilityLayer} +"

    return edgeMass

In [24]:
def Mass_Combination(compDisconnected, compConnected, combination, probabilityLayer):
    edgeVertexRelation = Edge_Vertex_Relation()
    
    cellMassDown = ""
    vertexIndexSCCDown = Same_SCC(compDisconnected[0].a, 0)

    for edgeIndex in range(4):
        cellMassDown += Edge_Mass_Contribution(edgeVertexRelation[edgeIndex], vertexIndexSCCDown, combination[edgeIndex], probabilityLayer[edgeIndex//2])

    cellMassDown = cellMassDown[:-2]

    
    cellMassUp = ""
    vertexIndexSCCUp = Same_SCC(compDisconnected[0].a, 3)
    
    for edgeIndex in range(4):
        cellMassUp += Edge_Mass_Contribution(edgeVertexRelation[edgeIndex], vertexIndexSCCUp, combination[edgeIndex], probabilityLayer[edgeIndex//2])
        
    cellMassUp = cellMassUp[:-2]

    
    cellMass = ""
    vertexIndexSCC = Same_SCC(compConnected[0].a, 0)

    for edgeIndex in range(4):
        cellMass += Edge_Mass_Contribution(edgeVertexRelation[edgeIndex], vertexIndexSCC, combination[edgeIndex], probabilityLayer[edgeIndex//2])
        
    cellMass = cellMass[:-2]

    
    return cellMassDown, cellMassUp, cellMass

In [19]:
combinations = Combinations_Generator(C,L)
combination = combinations[0]
print(combination)
g = Cell_Generator(L,C, combination)
compDown = graph_tool.topology.label_out_component(g, 0, label=None)
compUp = graph_tool.topology.label_out_component(g, L*C+1, label=None)
compDisconnected, compConnected = Cell_SCC(g)
percolationType = Percolation_Check(compDown, compUp)
cellMassDown, cellMassUp, cellMass = Mass_Combination(compDisconnected, compConnected, combination, probabilityLayer)
print(compDisconnected[0].a)
print(compConnected[0].a)
print(cellMassDown)
print(cellMassUp)
print(cellMass)

[0 0 0 0]
[0 1 2 3]
[0 1 2 0]
t_top +t_top
t_bottom +t_bottom
t_top +t_top +t_bottom +t_bottom


In [25]:
for combination in Combinations_Generator(C,L):
    g = Cell_Generator(L,C, combination)
    compDown = graph_tool.topology.label_out_component(g, 0, label=None)
    compUp = graph_tool.topology.label_out_component(g, L*C+1, label=None)
    percolationType = Percolation_Check(compDown, compUp)
    compDisconnected, compConnected = Cell_SCC(g)
    cellMassDown, cellMassUp, cellMass = Mass_Combination(compDisconnected, compConnected, combination, probabilityLayer)

In [26]:
functionMass = ""
functionttop = ""
functiontbottom = ""

functionm1 = ""
functionmi1top = ""
functionmi1bottom = ""

functionm2 = ""
functionmi2top = ""
functionmi2bottom = ""

for combination in Combinations_Generator(C, L):
    combinationProbability = Combination_Probability(L, C, combination, probabilityLayer)
    g = Cell_Generator(L, C, combination)
    compDown = graph_tool.topology.label_out_component(g, 0, label=None)
    compUp = graph_tool.topology.label_out_component(g, L * C + 1, label=None)
    percolationType = Percolation_Check(compDown, compUp)
    compDisconnected, compConnected = Cell_SCC(g)
    cellMassDown, cellMassUp, cellMass = Mass_Combination(compDisconnected, compConnected, combination, probabilityLayer)
    
    if percolationType == 0:
        functionttop += f"{combinationProbability}({cellMassDown})+"
        functiontbottom += f"{combinationProbability}({cellMassUp})+"
    elif percolationType == 1:
        functionm1 += f"{combinationProbability}({cellMass})+"
        functionmi1top += f"{combinationProbability}({cellMassDown})+"
        functionmi1bottom += f"{combinationProbability}({cellMassUp})+"
    elif percolationType == 2:
        functionm2 += f"{combinationProbability}({cellMass})+"
        functionmi2top += f"{combinationProbability}({cellMassDown})+"
        functionmi2bottom += f"{combinationProbability}({cellMassUp})+"
    elif percolationType == 3:
        functionMass += f"{combinationProbability}({cellMass})+"


In [28]:
functionMass

'pA0 pA3 pB0 pB3 (tAtop +MA +tBbottom +MB)+pA0 pA3 pB3 pB3 (tAtop +tAbottom +MA +MB +MB)+pA3 pA0 pB3 pB0 (MA +tAtop +MB +tBbottom)+pA3 pA0 pB3 pB3 (MA +tAtop +tAbottom +MB +MB)+pA3 pA3 pB0 pB3 (MA +MA +tBtop +tBbottom +MB)+pA3 pA3 pB3 pB0 (MA +MA +MB +tBtop +tBbottom)+pA3 pA3 pB3 pB3 (MA +MA +MB +MB)+'

In [30]:
functionttop

'pA0 pA0 pB0 pB0 (tAtop +tAtop)+pA0 pA0 pB0 pB3 (tAtop +tAtop)+pA0 pA0 pB3 pB0 (tAtop +tAtop)+pA0 pA0 pB3 pB3 (tAtop +tAtop)+pA0 pA3 pB0 pB0 (tAtop +MA +tBtop)+pA0 pA3 pB3 pB0 (tAtop +MA +tBtop)+pA3 pA0 pB0 pB0 (MA +tAtop +tBtop)+pA3 pA0 pB0 pB3 (MA +tAtop +tBtop)+pA3 pA3 pB0 pB0 (MA +MA +tBtop +tBtop)+'

In [31]:
functiontbottom

'pA0 pA0 pB0 pB0 (tBbottom +tBbottom)+pA0 pA0 pB0 pB3 (tAbottom +tBbottom +MB)+pA0 pA0 pB3 pB0 (tAbottom +MB +tBbottom)+pA0 pA0 pB3 pB3 (tAbottom +tAbottom +MB +MB)+pA0 pA3 pB0 pB0 (tBbottom +tBbottom)+pA0 pA3 pB3 pB0 (tAbottom +MB +tBbottom)+pA3 pA0 pB0 pB0 (tBbottom +tBbottom)+pA3 pA0 pB0 pB3 (tAbottom +tBbottom +MB)+pA3 pA3 pB0 pB0 (tBbottom +tBbottom)+'

In [36]:
#functionm1

In [38]:
#functionmi1top

In [40]:
#functionmi1bottom

In [42]:
#functionm2

In [44]:
#functionmi2top

In [27]:
functionmi2bottom

''